In [25]:
#all imports
import requests
import pandas as pd
from pathlib import Path
import numpy as np
import os

In [26]:
idx = pd.IndexSlice  #magically make MultiIndex slicing better

In [27]:
#create and reduce list of countries
#countries=pd.read_excel('raw_data/UNSD — Methodology.xlsx', sheetname='Sheet1')
#countries=countries.dropna(axis=0, subset=['ISO-alpha3 Code'])

In [28]:
#load original data
trade_data=pd.read_csv("raw_data/DataJobID-1257172_1257172_TestQuery.csv" , encoding = "ISO-8859-1")

In [29]:
#create data structures for building multi index dataframe
years = list(range(1995,2016))
countries=list(trade_data['ReporterName'].unique())

In [30]:
#TODO remove these
# countries_to_remove=['Belgium-Luxembourg', 'European Union', 'Other Asia, nes',' World']
# for country in countries_to_remove:
#     trade_data_export = trade_data_export[trade_data_export.Reporter != country]

In [31]:
def build_multi_index_df(years, countries):
    """
    Function which builds multi index dataframe suitable for this analysis.
    MultiIndex array is three-dimensional: year*country*country
    """ 

    #build MultiIndex array
    rows_array = []
    for year in years:
        for country in countries:
            rows_array.append([year,country])
        
    multi_index = pd.MultiIndex.from_tuples(rows_array, names=['year', 'exporter'])
    data = pd.DataFrame(columns=countries, index=multi_index)
    return data



In [32]:
#create dataframe structure
data = build_multi_index_df(years,countries)

In [33]:
%%timeit -n1 -r1

#fill dataframe with values from data
#Caution, takes roughly 6-8 minutes!
for index, row in trade_data.iterrows():
    for year in years:
        year_key=str(year)+" in 1000 USD "
        data.loc[year][row['ReporterName']][row['PartnerName']]=row[year_key]


1 loop, best of 1: 6min 51s per loop


In [34]:
#fill NaNs
data_filled=data.fillna(0)

In [35]:
#write to TSV
data_filled.to_csv('trade_data.tsv', sep='\t')

In [36]:
#read from TSV
imported_data = pd.read_table('trade_data.tsv', index_col=[0,1])

In [37]:
#compare whether identical
all(imported_data == data_filled)

True

In [38]:
imported_data.loc[2014]


,Aruba,Afghanistan,Angola,Anguila,Albania,Andorra,Netherlands Antilles,United Arab Emirates,Argentina,Armenia,...,United States,St. Vincent and the Grenadines,Venezuela,Vietnam,Vanuatu,Samoa,Yemen,South Africa,Zambia,Zimbabwe
exporter,,,,,,,,,,,,,,,,,,,,,
Aruba,0.000,0.0,0.000,0.0,0.000,0.504,0.0,0.634,1.403826e+03,0.000,...,1.257918e+06,0.0,0.0,0.000000e+00,0.0,0.000,0.000,445.114,0.000,0.000
Afghanistan,0.000,0.0,0.000,0.0,0.209,0.811,0.0,224351.287,1.228533e+03,251.700,...,6.775918e+05,0.0,0.0,0.000000e+00,0.0,0.000,0.000,1709.922,0.000,0.000
Angola,0.000,0.0,0.000,0.0,0.000,0.000,0.0,109504.917,1.269333e+05,13.447,...,1.875505e+06,0.0,0.0,8.143160e+04,0.0,0.000,0.000,1050602.388,8100.982,0.000
Anguila,0.000,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.000000e+00,0.000,...,6.002909e+04,0.0,0.0,0.000000e+00,0.0,0.000,0.000,70.657,0.000,0.000
Albania,0.000,0.0,0.000,0.0,0.000,4.039,0.0,3754.747,2.448014e+03,0.250,...,4.911402e+04,0.0,0.0,0.000000e+00,0.0,0.000,0.000,297.116,0.000,0.000
Andorra,0.000,0.0,0.000,0.0,0.000,0.000,0.0,59.045,0.000000e+00,4.600,...,3.570414e+03,0.0,0.0,3.520000e-01,0.0,0.000,9.802,32.200,0.000,4.000
Netherlands Antilles,0.000,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.000000e+00,0.000,...,0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.000,0.000,0.000,0.000,0.000
United Arab Emirates,0.000,0.0,867779.323,0.0,1543.602,122.881,0.0,0.000,2.001343e+05,8934.499,...,1.873584e+07,0.0,0.0,4.626999e+06,0.0,0.000,23000.439,1387348.744,230862.607,96085.818
Argentina,0.000,0.0,0.000,0.0,63.774,49.995,0.0,8891.343,0.000000e+00,0.300,...,9.523592e+06,0.0,0.0,1.740000e+05,0.0,0.000,131.639,213669.765,0.000,2.048
